In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
# pygeos is required to use geopandas spatial indexing
gpd.options.use_pygeos = True

from scripts import dl_utils

In [ ]:
fires = gpd.read_file('/Users/ckruse/Downloads/all_indonesia_fires.geojson')
fires.head()

In [ ]:
sum(fires['SATELLITE'] != 'Terra')

In [ ]:
fires['CONFIDENCE'][fires['CONFIDENCE'] == 'n'] = '50'
fires['CONFIDENCE'][fires['CONFIDENCE'] == 'l'] = '0'
fires['CONFIDENCE'][fires['CONFIDENCE'] == 'h'] = '100'
fires['CONFIDENCE'] = fires['CONFIDENCE'].astype(int)
fires['CONFIDENCE'].unique()

In [ ]:
sites = gpd.read_file('../../data/sampling_locations/gpw_se_asia_01-2019-06-2021_confirmed_positives.geojson')

In [ ]:
fires_index = fires['geometry'].sindex

In [ ]:
site_fires = []
for candidate in sites['geometry'].buffer(0.002):
    query = fires_index.query(candidate)
    site_fires.append(query)

In [ ]:
#sites['fire_index'] = site_fires
sites['fire_count'] = [len(index) for index in site_fires]
sites.sort_values(by='fire_count', ascending=False)
print(sites['fire_count'].sum())
sites.to_file('/Users/ckruse/Downloads/site_fires.geojson')

In [ ]:
fire_indices = []
for candidate in sites['geometry'].buffer(0.002):
    query = fires_index.query(candidate)
    if len(query) > 0:
        for val in query:
            fire_indices.append(val)

In [ ]:
fires['CONFIDENCE'][fire_indices].hist()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
patches = []
locations = []
dates = []
for index, site in enumerate(tqdm(site_fires)):
    if len(site) > 0:
        site_location = sites['geometry'][index]
        fire_locations = [fires['geometry'][fire_id]for fire_id in site]
        fire_dates = [fires['ACQ_DATE'][fire_id]for fire_id in site]
        fire_sensors = [fires['INSTRUMENT'][fire_id] for fire_id in site]
        for fire_location, fire_date, sensor in zip(tqdm(fire_locations), fire_dates, fire_sensors):
            #print(fire_date, fire_location.distance(site_location) * 111.1 * 1000)
            start = datetime.date.fromisoformat(fire_date) - relativedelta(days=3)
            end = start + relativedelta(days=3)
            try:
                patch, raster_info = dl_utils.download_patch(dl_utils.rect_from_point([fire_location.x, fire_location.y], 0.05), start.isoformat(), end.isoformat())
                patches.append(patch[0])
                locations.append(fire_location)
                dates.append(fire_date)
                plt.figure(figsize=(5,5), dpi=150)
                plt.imshow(np.clip(patch[0][:,:,3:0:-1] / 3000, 0, 1))
                plt.axis('off')
                title = f"Fire {start}, S2 {raster_info[0]['metadata']['']['id'][15:25]}, {sensor}"
                plt.title(title)
                plt.savefig(f'../../figures/fires/{title}-{fire_location.x:.4f}, {fire_location.y:.4f}.png', bbox_inches='tight')
                plt.close()
            except Exception as e:
                if e == KeyboardInterrupt:
                    break
                else:
                    print(f"No Scene Found\nDate: {start.isoformat()}-{end.isoformat()}")
        #print(geom.x, geom.y)
        #print(fires.iloc[index])
        #print(fire_locations)

In [ ]:
sites.sort_values(by='fire_count', ascending=False).head(50)

In [ ]:
dist = gpd.read_file('../../data/site_metadata/se_asia_metadata_2.0.geojson')['waterway distance']
sum(dist < 150) / len(dist)

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta

delta = relativedelta(days=1)
start = datetime.date.fromisoformat(fire_date)
end = start + delta
print(start, end)

In [ ]:
sites.to_file('/Users/ckruse/Downloads/site_fires.geojson')